<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-11 11:26:59
-------------------
qualified stocks: 93
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  2.51 L
Current:  1.32 C
-------------------
Today PnL: -12.98 K (-0.1%)
Current PnL: -22.73 L (-15.46%)
CY Booked + Current PnL: -14.46 L (-9.84%)
-------------------
Total profit:  2.08 L
Total loss:  -24.81 L
-------------------
Total Booked + Current PnL: 12.52 L (10.43%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.26%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 82.21 L (62.21%)
Deployed:  1.20 C
Current:  1.32 C
CAGR/XIRR %: 6.18%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SBIN,760.30,863.00,-9.61,M-LC,4.01,217923.0,16444.0,10765.0,2.24,8.16,4.94,13.51,62.0,1.53,1.68,21.79,XY25,NTT,BANKS
55,MEDANTA,1087.93,1486.00,25.69,H-SC,13.90,161008.0,34808.0,11367.0,-2.47,27.58,7.06,36.59,159.0,3.06,1.24,43.55,XY24,NTT,HEALTHCARE
36,HINDALCO,651.95,756.01,-21.55,H-LC,5.73,107952.0,3640.0,13008.0,0.29,3.49,12.05,15.96,12.0,0.28,0.83,20.98,X5K,ATH,METALS
83,TTKPRESTIG,769.29,770.00,-8.47,M-SC,10.94,83571.0,-17206.0,17299.0,0.65,-17.07,20.70,0.09,228.0,-0.99,0.64,9.79,OX40N,NTT,DURABLES
51,LALPATHLAB,2838.53,3545.00,-22.98,H-SC,8.66,219144.0,26124.0,21914.0,1.02,13.53,10.00,24.89,137.0,1.19,1.69,40.04,X40N,NTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,ICICIPRULI,600.83,790.00,-15.20,H-MC,2.11,136933.0,1145.0,41600.0,-0.68,0.84,30.38,31.48,110.0,0.03,1.06,13.06,X40,ATH,INSURANCE
25,DABUR,505.20,735.00,-8.98,H-MC,4.26,197262.0,234.0,89379.0,-0.29,0.12,45.31,45.49,101.0,0.00,1.52,13.17,XY24,BTT,FMCG
58,PGHH,13388.00,18062.58,-21.00,H-MC,6.54,199410.0,-1410.0,71528.0,0.54,-0.70,35.87,34.92,86.0,-0.02,1.54,3.94,X40,ATH,FMCG


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-21.82,H-LC,2.00,206075.0,1575.0,39834.0,1.24,0.77,19.33,20.25,10.0,0.04,1.59,9.57,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-14.80,H-LC,2.42,169286.0,-32100.0,119719.0,0.56,-15.94,70.72,43.51,28.0,-0.27,1.31,17.84,X40,BTT,PAINTS
53,LTIM,5564.16,7230.2,-5.95,H-LC,3.59,181062.0,-19248.0,79233.0,0.35,-9.61,43.76,29.94,24.0,-0.24,1.40,25.09,X200,ATH,IT
57,NESTLEIND,2268.60,2755.0,-59.41,H-LC,3.95,128911.0,-136515.0,193431.0,0.48,-51.43,150.05,21.44,8.0,-0.71,0.99,4.26,XY25,NTT,FMCG
10,BAJAJHFL,122.26,152.0,-11.24,H-LC,3.97,184238.0,-16268.0,65036.0,-0.03,-8.11,35.30,24.33,29.0,-0.25,1.42,6.24,X40N,BTT,FINANCE


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,KPIGREEN,497.21,731.35,32.29,H-SC,13.25,135526.0,10229.0,48776.0,6.23,8.16,35.99,47.09,148.0,0.21,1.05,69.62,MH,ATH,POWER
79,TATAMOTORS,761.77,1065.00,-45.88,M-LC,5.52,236803.0,-39720.0,149802.0,2.94,-14.36,63.26,39.81,54.0,-0.27,1.83,13.49,XY24,NTT,AUTO
71,SIEMENS,4406.90,7969.85,-7.83,H-LC,36.10,77872.0,-32300.0,121371.0,2.74,-29.32,155.86,80.85,19.0,-0.27,0.60,14.69,AR,ATH,ELECTRICAL
68,SBIN,760.30,863.00,-9.61,M-LC,4.01,217923.0,16444.0,10765.0,2.24,8.16,4.94,13.51,62.0,1.53,1.68,21.79,XY25,NTT,BANKS
20,CERA,7989.07,9475.00,-20.29,H-SC,3.06,113823.0,-29980.0,56729.0,1.90,-20.85,49.84,18.60,151.0,-0.53,0.88,25.61,OX40N,NTT,CERAMICS


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,INDIGOPNTS,1407.73,1408.00,104.80,M-SC,3.71,134441.0,-40118.0,40158.0,-5.92,-22.98,29.87,0.02,224.0,-1.00,1.04,16.58,OX40N,NTT,PAINTS
90,VOLTAS,1278.28,1918.49,-31.94,H-MC,8.11,185430.0,-6312.0,102339.0,-5.26,-3.29,55.19,50.08,89.0,-0.06,1.43,3.28,XY25,ATH,AC
52,LAOPALA,369.40,464.00,104.98,H-SC,3.16,65110.0,-35736.0,61562.0,-3.60,-35.44,94.55,25.61,147.0,-0.58,0.50,20.72,AR,NTT,CERAMICS
84,UJJIVANSFB,52.77,60.00,31.86,H-SC,20.58,114858.0,-27621.0,47138.0,-3.54,-19.39,41.04,13.70,161.0,-0.59,0.89,35.56,OX40N,NTT,BANKS
29,EASEMYTRIP,18.12,26.40,-10.77,M-SC,22.95,85870.0,-90945.0,171740.0,-3.40,-51.44,200.00,45.70,199.0,-0.53,0.66,0.00,XY24,NTT,TRAVEL


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,-8.47,M-SC,10.94,83571.0,-17206.0,17299.0,0.65,-17.07,20.70,0.09,228.0,-0.99,0.64,9.79,OX40N,NTT,DURABLES
84,UJJIVANSFB,52.77,60.0,31.86,H-SC,20.58,114858.0,-27621.0,47138.0,-3.54,-19.39,41.04,13.70,161.0,-0.59,0.89,35.56,OX40N,NTT,BANKS
20,CERA,7989.07,9475.0,-20.29,H-SC,3.06,113823.0,-29980.0,56729.0,1.90,-20.85,49.84,18.60,151.0,-0.53,0.88,25.61,OX40N,NTT,CERAMICS
72,SIS,477.00,528.0,2246.11,H-SC,7.07,63121.0,-17969.0,26637.0,-0.39,-22.16,42.20,10.69,157.0,-0.67,0.49,27.92,OX40N,NTT,MISC
78,TATAELXSI,7332.28,9161.0,-24.03,H-MC,2.35,79737.0,-22915.0,48520.0,-1.11,-22.32,60.85,24.94,98.0,-0.47,0.62,21.21,OX40N,NTT,IT


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-27.06,H-LC,15.51,237705.0,-58229.0,152131.0,0.37,-19.68,64.00,31.73,2.0,-0.38,1.83,1.48,X40,BTT,IT
89,VBL,492.64,671.64,-10.47,H-LC,10.94,271636.0,6596.0,89694.0,1.80,2.49,33.02,36.33,3.0,0.07,2.10,16.12,X40N,ATH,FMCG
45,INFY,1461.46,2275.00,-20.32,H-LC,9.83,255379.0,-6222.0,151848.0,0.25,-2.38,59.46,55.67,4.0,-0.04,1.97,3.58,X40,BTT,IT
1,ABB,5551.76,7934.00,-44.34,H-LC,9.69,198724.0,-17795.0,110709.0,1.46,-8.22,55.71,42.91,5.0,-0.16,1.53,4.07,AR,NTT,ELECTRICAL
57,NESTLEIND,2268.60,2755.00,-59.41,H-LC,3.95,128911.0,-136515.0,193431.0,0.48,-51.43,150.05,21.44,8.0,-0.71,0.99,4.26,XY25,NTT,FMCG


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,25.69,H-SC,13.9,161008.0,34808.0,11367.0,-2.47,27.58,7.06,36.59,159.0,3.06,1.24,43.55,XY24,NTT,HEALTHCARE


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,KPIGREEN,497.21,731.35,32.29,H-SC,13.25,135526.0,10229.0,48776.0,6.23,8.16,35.99,47.09,148.0,0.21,1.05,69.62,MH,ATH,POWER
55,MEDANTA,1087.93,1486.00,25.69,H-SC,13.90,161008.0,34808.0,11367.0,-2.47,27.58,7.06,36.59,159.0,3.06,1.24,43.55,XY24,NTT,HEALTHCARE
35,HEROMOTOCO,4311.81,5949.07,-0.95,H-MC,6.54,159502.0,8589.0,48712.0,-0.93,5.69,30.54,37.97,96.0,0.18,1.23,32.44,AR,ATH,AUTO
15,BLUESTARCO,1646.70,2326.38,-6.04,H-SC,7.66,176310.0,11640.0,56331.0,-2.07,7.07,31.95,41.28,124.0,0.21,1.36,15.62,X40N,ATH,AC
19,CAMS,3643.00,5211.76,-7.25,H-SC,2.69,104115.0,2111.0,41813.0,-0.71,2.07,40.16,43.06,122.0,0.05,0.80,19.54,X40N,ATH,MISC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,KPIGREEN,497.21,731.35,32.29,H-SC,13.25,135526.0,10229.0,48776.0,6.23,8.16,35.99,47.09,148.0,0.21,1.05,69.62,MH,ATH,POWER
35,HEROMOTOCO,4311.81,5949.07,-0.95,H-MC,6.54,159502.0,8589.0,48712.0,-0.93,5.69,30.54,37.97,96.0,0.18,1.23,32.44,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-48.26,H-SC,12.85,134578.0,11242.0,122520.0,-0.86,9.12,91.04,108.45,119.0,0.09,1.04,33.51,AR,ATH,MISC
91,WHIRLPOOL,1219.98,2270.00,-43.86,M-SC,4.47,94545.0,3046.0,75702.0,-0.32,3.33,80.07,86.07,212.0,0.04,0.73,36.77,XR,NTT,DURABLES
92,WIPRO,243.46,420.00,-12.85,M-LC,6.68,149346.0,-1599.0,111054.0,0.77,-1.06,74.36,72.51,55.0,-0.01,1.15,4.90,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-53.72,L-SC,22.24,68832.0,-44717.0,84767.0,-0.55,-39.38,123.15,35.27,267.0,-0.53,0.53,68.47,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-4.34,H-SC,17.40,89079.0,-11884.0,106414.0,0.36,-11.77,119.46,93.63,150.0,-0.11,0.69,32.68,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-20.96,H-SC,15.84,89790.0,-8190.0,30062.0,-1.03,-8.36,33.48,22.32,152.0,-0.27,0.69,30.23,XR,ATH,FINANCE
36,HINDALCO,651.95,756.01,-21.55,H-LC,5.73,107952.0,3640.0,13008.0,0.29,3.49,12.05,15.96,12.0,0.28,0.83,20.98,X5K,ATH,METALS
84,UJJIVANSFB,52.77,60.00,31.86,H-SC,20.58,114858.0,-27621.0,47138.0,-3.54,-19.39,41.04,13.70,161.0,-0.59,0.89,35.56,OX40N,NTT,BANKS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
89,VBL,492.64,671.64,-10.47,H-LC,10.94,271636.0,6596.0,89694.0,1.80,2.49,33.02,36.33,3.0,0.07,2.10,16.12,X40N,ATH,FMCG
70,SHALBY,261.39,327.00,933.49,M-SC,7.99,140485.0,-40658.0,86131.0,-0.27,-22.45,61.31,25.10,236.0,-0.47,1.08,12.58,XY24,NTT,HEALTHCARE
51,LALPATHLAB,2838.53,3545.00,-22.98,H-SC,8.66,219144.0,26124.0,21914.0,1.02,13.53,10.00,24.89,137.0,1.19,1.69,40.04,X40N,NTT,HEALTHCARE
35,HEROMOTOCO,4311.81,5949.07,-0.95,H-MC,6.54,159502.0,8589.0,48712.0,-0.93,5.69,30.54,37.97,96.0,0.18,1.23,32.44,AR,ATH,AUTO
55,MEDANTA,1087.93,1486.00,25.69,H-SC,13.90,161008.0,34808.0,11367.0,-2.47,27.58,7.06,36.59,159.0,3.06,1.24,43.55,XY24,NTT,HEALTHCARE


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.48
1,25,41.13
2,50,70.87


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    41.24
LC    36.61
MC    22.13
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.98
X40N     15.35
X40      13.44
XY25     10.70
XR        8.96
AR        8.42
OX40N     7.56
X5K       2.23
MH        1.81
X200      1.40
SR        1.13
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.30
H-SC    26.37
H-MC    19.17
M-SC    13.43
M-LC     7.23
M-MC     2.61
L-SC     1.44
L-LC     1.08
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.33,-10.96,46.63
IT,12.02,-19.80,79.64
FINANCE,8.31,-20.87,65.59
BANKS,7.65,-15.04,63.89
MISC,5.92,-19.97,88.74
PAINTS,5.66,-19.00,42.45
HEALTHCARE,5.22,3.08,21.39
ELECTRICAL,4.73,-14.35,63.87
AUTO,4.44,-17.50,68.62


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2878832.0,23
XR,1131695.0,13
AR,1116780.0,10
X40,829454.0,10
XY25,660094.0,8
X40N,618317.0,12
OX40N,479350.0,10
SR,196992.0,2
X5K,148956.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2828902.0,29
M-SC,1503078.0,17
H-LC,1484310.0,20
H-MC,1245613.0,15
M-LC,441163.0,5
M-MC,339219.0,2
L-SC,274219.0,3
L-MC,57774.0,1
L-LC,46263.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      956585.0      7
M-SC       XY24      878012.0      7
H-SC       AR        674389.0      5
H-LC       X40       548157.0      6
H-SC       XR        533081.0      6
H-MC       XY24      477245.0      4
M-MC       XY24      339219.0      2
H-LC       X40N      313271.0      6
           XY25      254186.0      3
           AR        232080.0      2
H-MC       XY25      223482.0      2
H-SC       X40N      215091.0      4
H-MC       X40       205829.0      3
H-SC       OX40N     203988.0      4
           SR        196992.0      2
L-SC       XR        183542.0      2
M-LC       XY24      183396.0      2
M-SC       AR        161599.0      2
           OX40N     136165.0      4
M-LC       X5K       135948.0      1
M-SC       XR        126436.0      2
           XY25      125398.0      1
H-MC       XR        119808.0      1
M-LC       XR        111054.0      1
L-SC       OX40N      90677.0      1
H-MC       X40N       89955.0      2
H-LC       X200       79233.0      1
M-SC       X40        75468.0      1
L-MC       XR         57774.0      1
H-SC       MH         48776.0      1
H-MC       AR         48712.0      1
           OX40N      48520.0      1
L-LC       XY25       46263.0      1
H-LC       XY24       44375.0      1
H-MC       MH         32062.0      1
H-LC       X5K        13008.0      1
M-LC       XY25       10765.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
